# 2.1 神经元（下）

在上一节中，我们用反向传播配合激活函数实现了一个简单的神经网络。在本节，我们将探索神经网络更具有泛化的模型编写，同时实现更多基于pytorch这一由Meta（Facebook母公司）开发的深度学习框架的底层实现（这个框架的所有算法都由c/c++编写，再跟python代码绑定），我们将重点实现这一框架独有的动态图，这一有别于其余深度学习框架（Tensorflow）的静态图的独特实现。

## 数字
首先我们实现一个数字类，这个类有一个更独特的学名：张量（Tensor）

In [1]:
class Tensor:
    def __init__(self, data):
        self.data = data

    def __add__(self, other):
        out = Tensor(self.data + other.data)
        return out

    def __mul__(self, other):
        out = Tensor(self.data * other.data)
        return out
    
    def __repr__(self):
        return f"Tensor(data={self.data})"

我们定义了一个名叫Tensor的类。实现了针对加法和乘法的运算符重载，同时我们还实现了魔术方法\_\_repr\_\_用于打印我们的结果，我们来测试一下这个类：

In [2]:
a = Tensor(12)
b = Tensor(15)
print(a + b)
print(a * b)

Tensor(data=27)
Tensor(data=180)
